In [ ]:
import requests
import time
import pandas as pd
import numpy as np
from web3 import Web3

pd.options.mode.chained_assignment = None

## Etherscan

In [ ]:
FOLDER = './'

with open(FOLDER + 'etherscan.txt', 'r') as file:
    KEY = file.read()





In [ ]:
df = pd.read_csv(FOLDER + 'blocks.csv')

In [ ]:
def build_payload(block):
    PAYLOAD = 'https://api.etherscan.io/api' \
             + '?module=block' \
             + '&action=getblockreward' \
             + f'&blockno={block}' \
             + f'&apikey={KEY}'
    return PAYLOAD

In [ ]:
def send_payload(payload):
    try:
        response = requests.get(payload)
        raw = response.json()
        raw = raw['result']
    except:
        time.sleep(5)
        return send_payload(payload)

    return raw


In [ ]:
for i in range(0, len(df)):
    block = df['block_number'].iloc[i]
    payload = build_payload(block)
    raw = send_payload(payload)
    processed_df = process_payload(raw)
    df = merge_tables(df, processed_df)




In [ ]:
def process_payload(raw):
    raw.pop('uncles', None)
    raw.pop('uncleInclusionReward', None)
    
    processed_df = pd.DataFrame(raw, index=[0])
    
    processed_df['block_number'] = processed_df['block_number'].astype(int)
    processed_df['timestamp'] = processed_df['timestamp'].astype(int)
    processed_df['blockReward'] = processed_df['blockReward'].astype(int)
    
    raw_df.rename(columns={'blockNumber': 'block_number', 'timeStamp': 'timestamp'}, inplace=True)
    
    return processed_df

In [ ]:
def merge_tables(df, processed_df):

    df = df.merge(raw_df, how='left', on='block_number')
    
    return df


In [ ]:
df.to_csv(FOLDER + 'blocks.csv', index=False)

## Infura

In [ ]:
FOLDER = './'

with open(FOLDER + 'infura.txt', 'r') as file:
    RPC_ENDPOINT = file.read()

In [ ]:
w3 = Web3(Web3.HTTPProvider(RPC_ENDPOINT))
assert w3.isConnected()

for i in range(0, len(df)):
    block = df['block_number'].iloc[i]

    block_dct = send_request(block)
    block_df = process_block(block_dct)
    df = merge_tables(df, block_df)

In [ ]:
def send_request(block):
    try:
        block_dct = w3.eth.getBlock(block)
    except:
        time.sleep(5)
        return send_request(block)
    
    return block_dct

In [ ]:
def process_block(block_dct):

    block_df = pd.DataFrame(columns = ['block_number', 'block_hash', 'txn_hashes', 'txn_count'], index=[0])
    block_df['block_number'] = block_dct['number']
    block_df['block_hash'] = block_dct['hash'].hex()

    txn_hashes = [tx.hex() for tx in block_dct['transactions']]
    block_df['txn_hashes'][0] = list(txn_hashes)
    block_df['txn_count'] = len(txn_hashes)

    return block_df


In [ ]:
def merge_tables(df, block_df):

    df = df.merge(block_df, how='left', on='block_number')
    
    return df


In [ ]:
df.to_csv(FOLDER + 'blocks.csv', index=False)